In [ ]:
#%matplotlib widget

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#%pip install lab_black
#%load_ext lab_black
%cd ..

In [ ]:
!ls

In [ ]:
from pathlib import Path

import torch as th
import torch.nn.functional as F
import numpy as np
import yaml
from easydict import EasyDict

from src.utils import instantiate_from_config, get_device
from src.utils.vis import save_sdf_as_mesh

In [ ]:
th.set_grad_enabled(False)
device = get_device()

    th.cuda.set_device(device)
device

# Load Pretrained Models

In [ ]:
gen32_args_path = "config/gen32/chair.yaml"
gen32_ckpt_path = "results/gen32/chair.pth"
sr64_args_path = "config/sr32_64/chair.yaml"
sr64_ckpt_path = "results/sr32_64/chair.pth"

In [ ]:
with open(gen32_args_path) as f:
    args1 = EasyDict(yaml.safe_load(f))
with open(sr64_args_path) as f:
    args2 = EasyDict(yaml.safe_load(f))

In [ ]:
model1 = instantiate_from_config(args1.model)
model1 = model1.cuda() if th.cuda.is_available() else model1
ckpt = th.load(gen32_ckpt_path, map_location="cpu")
model1.load_state_dict(ckpt["model_ema"])

In [ ]:
model2 = instantiate_from_config(args2.model)
model2 = model2.cuda() if th.cuda.is_available() else model2
ckpt = th.load(sr64_ckpt_path, map_location="cpu")
model2.load_state_dict(ckpt["model"])

In [ ]:
ddpm_sampler1 = instantiate_from_config(args1.ddpm.valid)
ddpm_sampler2 = instantiate_from_config(args2.ddpm.valid)

if th.cuda.is_available():
    ddpm_sampler1, ddpm_sampler2 = ddpm_sampler1.cuda(), ddpm_sampler2.cuda()

In [ ]:
preprocessor1 = instantiate_from_config(args1.preprocessor, device)
preprocessor2 = instantiate_from_config(args2.preprocessor, device)

# Generate Low-Resolution ($32^3$)

Generates 5 low-resolution samples

In [ ]:
out1 = ddpm_sampler1.sample_ddim(model1, (3, 1, 32, 32, 32), show_pbar=True)

In [ ]:
out1 = preprocessor1.destandardize(out1)
out1.shape

In [ ]:
!nvidia-smi

In [ ]:
from src.utils.vis import plot_sdfs
plot_sdfs(list(out1))

In [ ]:
# save as an obj file
for i, out in enumerate(out1):
    save_sdf_as_mesh(f"gen32_{i}-stokes.obj", out, safe=True)

In [ ]:
lr_cond = F.interpolate(out1, (64, 64, 64), mode="nearest")
lr_cond = preprocessor2.standardize(lr_cond, 0)
out2 = ddpm_sampler2.sample_ddim(lambda x, t: model2(th.cat([lr_cond, x], 1), t), (out1.shape[0], 1, 64, 64, 64), show_pbar=True)

out2 = preprocessor2.destandardize(out2, 1)

for i, out in enumerate(out2):
    save_sdf_as_mesh(f"sr64_{i}-stokes.obj", out, safe=True)

plot_sdfs(list(out2), title="Super-resolution origianal samples")

In [ ]:
# Test inversion
out1_inv = ddpm_sampler1.sample_ddim(model1, x_t = ddpm_sampler1.invert_ddim(model1, preprocessor1.standardize(out1), show_pbar=True, debug_plot=True), show_pbar=True)


In [ ]:
out1_inv = preprocessor1.destandardize(out1_inv)
out1_inv.shape

In [ ]:
# save as an obj file
for i, out in enumerate(out1_inv):
    save_sdf_as_mesh(f"inv_gen32_{i}-stokes.obj", out, safe=True)

In [ ]:
# compute norm difference
th.norm(out1 - out1_inv)

In [ ]:
plot_sdfs(out1_inv)

In [ ]:
plot_sdfs([out1, out1_inv], titles=["Original", "Predicted from inversion"])

In [ ]:
del out1_inv

In [ ]:
# Optimization 
def volume_estimates(sdfs, surface_threshold=1e-8): 
    # Get the dimensions of the SDF tensor
    sdf_resolution = sdfs.shape[-1]
    
    # Compute the voxel size
    # TODO remove voxel_size as we don't care about the scale
    voxel_size = 1.0 / (sdf_resolution - 1)
    
    # Create a grid of voxel coordinates
    x = th.linspace(0, 1, sdf_resolution)
    y = th.linspace(0, 1, sdf_resolution)
    z = th.linspace(0, 1, sdf_resolution)
    x, y, z = th.meshgrid(x, y, z)
    
    # th the position vector and normal vector
    r = th.stack([x, y, z], dim=-1)
    with th.enable_grad():
        sdfs.requires_grad_(True)
        n = th.autograd.grad(sdfs, [x, y, z], create_graph=True)[0]
        n = n / th.norm(n, dim=-1, keepdim=True)
    
    # Find the boundary voxels (where SDF is approximately 0)
    boundary_mask = th.isclose(sdfs, th.zeros_like(sdfs), atol=surface_threshold)
    boundary_indices = th.where(boundary_mask)

    
    # Compute the volume (Stokes' theorem)
    volume = th.sum(r[boundary_indices] * n[boundary_indices] * voxel_size ** 2)
    
    return volume.item()


def volume_estimates(sdfs, surface_threshold=1e-6): 
    # Get the size of the SDF tensor
    batch_size, _, depth, height, width = sdfs.shape

    # Compute the gradient of the SDF tensor
    sdfs.requires_grad_(True)
    n = th.autograd.grad(sdfs, (sdfs,), grad_outputs=th.ones_like(sdfs), create_graph=True)
    n = th.stack(n, dim=1)  # Shape: (batch_size, 3, depth, height, width)
    assert th.allclose(th.norm(n, dim=(-3, -2, -1)), 1)

    # Compute the boundaries with a Gaussian activation function 
    gaussian_activation = lambda x, sigma: th.exp(- (x / sigma) ** 2)
    sdfs_boundary = gaussian_activation(sdfs, surface_threshold)

    # Compute the volume using Stokes' Theorem
    volume = th.sum(n @ sdfs_boundary)



def volume_estimates_loss_fn(xs, target_volumes, max_volume=1., reduction="none"):
    input_volumes = volume_estimates(xs) / max_volume
    target_volumes = target_volumes / max_volume
    loss = th.nn.MSELoss(reduction=reduction)(input_volumes, target_volumes)
    return loss

In [ ]:
import torch as th

# Ensure the src module is in the path
import sys
sys.path.append('..')

from src.utils.vis import plot_sdfs

def generate_batched_sdf_tensor(batch_size, shape, centers, radii):
    x = th.linspace(0, 1, shape[0])
    y = th.linspace(0, 1, shape[1])
    z = th.linspace(0, 1, shape[2])
    X, Y, Z = th.meshgrid(x, y, z, indexing='ij')
    sdf_batch = th.zeros((batch_size, *shape))
    for i in range(batch_size):
        sdf_batch[i] = th.sqrt((X - centers[i][0])**2 + (Y - centers[i][1])**2 + (Z - centers[i][2])**2) - radii[i]
    return sdf_batch

xx = generate_batched_sdf_tensor(4, (10, 10, 10), [(0.0, 0.0, 0.0), (0.5, 0.5, 0.5)]*2, list(range(1, 5)))
xx.shape

In [ ]:
plot_sdfs(xx.unsqueeze(1))

In [ ]:
xx.requires_grad_(True)
nn = th.autograd.grad(xx, (xx,), grad_outputs=th.ones_like(xx), create_graph=True)
nn = th.stack(nn, dim=1)
nn.shape

***I don't think autograd can be used because there is no computational path between the input grid and the sdfs*** -- > gradient should be approximated with difference 

In [ ]:
th.autograd.grad(xx, th.meshgrid([th.linspace(0, 1, d) for d in xx.shape], indexing='ij'), create_graph=True)[0].shape

In [ ]:
xx.requires_grad_(False)
for g in th.gradient(xx, spacing=1/(xx.shape[-1]-1)): 
    print(g.shape)

# TODO.. 


In [ ]:
out1_std = preprocessor1.standardize(out1)

target_volume_increment = 0.4
t_optim_idx = 8
x_edited, x_t, x_t_optim = ddpm_sampler1.ddim_sample_latent_optimization(
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increment), "max_volume": out1.shape[-3:].numel()*preprocessor1.sdf_clip},
    t_optim_idx=t_optim_idx, 
    tgt_noise_level="t_optim"
).values()

plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[out1, preprocessor1.destandardize(x_edited), th.abs(out1- preprocessor1.destandardize(x_edited))], 
    title = f"Optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        [f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increment = 0.4
t_optim_idx = 5
x_edited, x_t, x_t_optim = ddpm_sampler1.ddim_sample_latent_optimization(
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increment), "max_volume": out1.shape[-3:].numel()*preprocessor1.sdf_clip},
    t_optim_idx=t_optim_idx, 
    tgt_noise_level="t_optim"
).values()

plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[out1, preprocessor1.destandardize(x_edited), th.abs(out1- preprocessor1.destandardize(x_edited))], 
    title = f"Optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        [f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increment = 0.4
t_optim_idx = 15
x_edited, x_t, x_t_optim = ddpm_sampler1.ddim_sample_latent_optimization(
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn, 
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increment), "max_volume": out1.shape[-3:].numel()*preprocessor1.sdf_clip},
    t_optim_idx=t_optim_idx, 
    tgt_noise_level="t_optim"
).values()

plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[out1, preprocessor1.destandardize(x_edited), th.abs(out1- preprocessor1.destandardize(x_edited))], 
    title = f"Optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        [f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increment = 0.4
t_optim_idx = 3
x_edited, x_t, x_t_optim = ddpm_sampler1.ddim_sample_latent_optimization(
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn,
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increment), "max_volume": out1.shape[-3:].numel()*preprocessor1.sdf_clip},
    t_optim_idx=t_optim_idx,
    tgt_noise_level="zero"
).values()

plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[out1, preprocessor1.destandardize(x_edited), th.abs(out1- preprocessor1.destandardize(x_edited))], 
    title = f"Optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        [f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
for i, out in enumerate(preprocessor1.destandardize(x_edited)):
    save_sdf_as_mesh(f"gen32_{i}_v40%incr_3steps-stokes.obj", out, safe=True)

In [ ]:
lr_cond_edit = F.interpolate(preprocessor1.destandardize(x_edited), (64, 64, 64), mode="nearest")
lr_cond_edit = preprocessor2.standardize(lr_cond_edit, 0)
x_edited_sr = ddpm_sampler2.sample_ddim(lambda x, t: model2(th.cat([lr_cond_edit, x], 1), t), (out1.shape[0], 1, 64, 64, 64), show_pbar=True)

x_edited_sr = preprocessor2.destandardize(x_edited_sr, 1)

for i, out in enumerate(x_edited_sr):
    save_sdf_as_mesh(f"sr64_{i}_v40%incr_3steps-stokes.obj", out, safe=True)

plot_sdfs(
    sdfs=[out2, x_edited_sr], 
    title = f"Super-resolution results of optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original (SR) ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited (SR) ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        ]
)

In [ ]:
target_volume_increment = 0.4
t_optim_idx = 5
x_edited, x_t, x_t_optim = ddpm_sampler1.ddim_sample_latent_optimization(
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn,
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increment), "max_volume": out1.shape[-3:].numel()*preprocessor1.sdf_clip},
    t_optim_idx=t_optim_idx,
    tgt_noise_level="zero"
).values()

plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[out1, preprocessor1.destandardize(x_edited), th.abs(out1- preprocessor1.destandardize(x_edited))], 
    title = f"Optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        [f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
# Check if it's better to don't use the normalized loss
target_volume_increment = 0.4
t_optim_idx = 5
x_edited, x_t, x_t_optim = ddpm_sampler1.ddim_sample_latent_optimization(
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn,
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increment), "max_volume": 1},
    t_optim_idx=t_optim_idx,
    tgt_noise_level="zero", 
    loss_threshold=10,
    opt_kwargs={"lr": 1e-2}
).values()

plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[out1, preprocessor1.destandardize(x_edited), th.abs(out1- preprocessor1.destandardize(x_edited))], 
    title = f"Optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        [f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
for i, out in enumerate(preprocessor1.destandardize(x_edited)):
    save_sdf_as_mesh(f"gen32_{i}_v40%incr_5steps-stokes.obj", out, safe=True)

lr_cond_edit = F.interpolate(preprocessor1.destandardize(x_edited), (64, 64, 64), mode="nearest")
lr_cond_edit = preprocessor2.standardize(lr_cond_edit, 0)
x_edited_sr = ddpm_sampler2.sample_ddim(lambda x, t: model2(th.cat([lr_cond_edit, x], 1), t), (out1.shape[0], 1, 64, 64, 64), show_pbar=True)

x_edited_sr = preprocessor2.destandardize(x_edited_sr, 1)

for i, out in enumerate(x_edited_sr):
    save_sdf_as_mesh(f"sr64_{i}_v40%incr_5steps-stokes.obj", out, safe=True)

plot_sdfs(
    sdfs=[out2, x_edited_sr], 
    title = f"Super-resolution results of optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original (SR) ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited (SR) ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        ]
)


In [ ]:
target_volume_increment = 0.4
t_optim_idx = 8
x_edited, x_t, x_t_optim = ddpm_sampler1.ddim_sample_latent_optimization(
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn,
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increment)},
    t_optim_idx=t_optim_idx,
    tgt_noise_level="zero"
)

plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[out1, preprocessor1.destandardize(x_edited), th.abs(out1- preprocessor1.destandardize(x_edited))], 
    title = f"Optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        [f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
for i, out in enumerate(preprocessor1.destandardize(x_edited)):
    save_sdf_as_mesh(f"gen32_{i}_v40%incr_8steps-stokes.obj", out, safe=True)

lr_cond_edit = F.interpolate(preprocessor1.destandardize(x_edited), (64, 64, 64), mode="nearest")
lr_cond_edit = preprocessor2.standardize(lr_cond_edit, 0)
x_edited_sr = ddpm_sampler2.sample_ddim(lambda x, t: model2(th.cat([lr_cond_edit, x], 1), t), (out1.shape[0], 1, 64, 64, 64), show_pbar=True)

x_edited_sr = preprocessor2.destandardize(x_edited_sr, 1)

for i, out in enumerate(x_edited_sr):
    save_sdf_as_mesh(f"sr64_{i}_v40%incr_8steps-stokes.obj", out, safe=True)

plot_sdfs(
    sdfs=[out2, x_edited_sr], 
    title = f"Super-resolution results of optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original (SR) ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited (SR) ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        ]
)

In [ ]:
target_volume_increment = -0.4
t_optim_idx = 3
x_edited, x_t, x_t_optim = ddpm_sampler1.ddim_sample_latent_optimization(
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn,
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increment)},
    t_optim_idx=t_optim_idx,
    tgt_noise_level="zero"
).values()

plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[out1, preprocessor1.destandardize(x_edited), th.abs(out1- preprocessor1.destandardize(x_edited))], 
    title = f"Optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        [f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
for i, out in enumerate(preprocessor1.destandardize(x_edited)):
    save_sdf_as_mesh(f"gen32_{i}_-v40%incr_3steps-stokes.obj", out, safe=True)

lr_cond_edit = F.interpolate(preprocessor1.destandardize(x_edited), (64, 64, 64), mode="nearest")
lr_cond_edit = preprocessor2.standardize(lr_cond_edit, 0)
x_edited_sr = ddpm_sampler2.sample_ddim(lambda x, t: model2(th.cat([lr_cond_edit, x], 1), t), (out1.shape[0], 1, 64, 64, 64), show_pbar=True)

x_edited_sr = preprocessor2.destandardize(x_edited_sr, 1)

for i, out in enumerate(x_edited_sr):
    save_sdf_as_mesh(f"sr64_{i}_-v40%incr_3steps-stokes.obj", out, safe=True)

plot_sdfs(
    sdfs=[out2, x_edited_sr], 
    title = f"Super-resolution results of optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original (SR) ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited (SR) ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        ]
)

In [ ]:
# Check if it's better to don't use the normalized loss
target_volume_increment = -0.4
t_optim_idx = 3
x_edited, x_t, x_t_optim = ddpm_sampler1.ddim_sample_latent_optimization(
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn,
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increment), "max_volume": 1},
    t_optim_idx=t_optim_idx,
    tgt_noise_level="zero", 
    loss_threshold=10,
    opt_kwargs={"lr": 1e-2}
).values()

plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[out1, preprocessor1.destandardize(x_edited), th.abs(out1- preprocessor1.destandardize(x_edited))], 
    title = f"Optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        [f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
target_volume_increment = -0.4
t_optim_idx = 5
x_edited, x_t, x_t_optim = ddpm_sampler1.ddim_sample_latent_optimization(
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn,
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increment)},
    t_optim_idx=t_optim_idx,
    tgt_noise_level="zero"
).values()

plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[out1, preprocessor1.destandardize(x_edited), th.abs(out1- preprocessor1.destandardize(x_edited))], 
    title = f"Optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        [f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)
for i, out in enumerate(preprocessor1.destandardize(x_edited)):
    save_sdf_as_mesh(f"gen32_{i}_-v40%incr_5steps-stokes.obj", out, safe=True)

lr_cond_edit = F.interpolate(preprocessor1.destandardize(x_edited), (64, 64, 64), mode="nearest")
lr_cond_edit = preprocessor2.standardize(lr_cond_edit, 0)
x_edited_sr = ddpm_sampler2.sample_ddim(lambda x, t: model2(th.cat([lr_cond_edit, x], 1), t), (out1.shape[0], 1, 64, 64, 64), show_pbar=True)

x_edited_sr = preprocessor2.destandardize(x_edited_sr, 1)

for i, out in enumerate(x_edited_sr):
    save_sdf_as_mesh(f"sr64_{i}_-v40%incr_5steps-stokes.obj", out, safe=True)

plot_sdfs(
    sdfs=[out2, x_edited_sr], 
    title = f"Super-resolution results of optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original (SR) ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited (SR) ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        ]
)

In [ ]:
# Check if it's better to don't use the normalized loss
target_volume_increment = -0.4
t_optim_idx = 5
x_edited, x_t, x_t_optim = ddpm_sampler1.ddim_sample_latent_optimization(
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn,
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increment), "max_volume": 1},
    t_optim_idx=t_optim_idx,
    tgt_noise_level="zero", 
    loss_threshold=10,
    opt_kwargs={"lr": 1e-2}
).values()

plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[out1, preprocessor1.destandardize(x_edited), th.abs(out1- preprocessor1.destandardize(x_edited))], 
    title = f"Optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        [f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)

In [ ]:
# Decrease the LR a bit
target_volume_increment = -0.4
t_optim_idx = 5
x_edited, x_t, x_t_optim = ddpm_sampler1.ddim_sample_latent_optimization(
    model1, 
    x_0=out1_std, 
    obj_fn=volume_estimates_loss_fn,
    obj_fn_args={"target_volumes": volume_estimates(out1_std) * (1+target_volume_increment), "max_volume": 1},
    t_optim_idx=t_optim_idx,
    tgt_noise_level="zero", 
    loss_threshold=10,
    opt_kwargs={"lr": .5e-2}
).values()

plot_sdfs(sdfs=[x_t, x_t_optim], title=f"Latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$", titles=["Latent to be optimized", "Optimized latent"]) 

plot_sdfs(
    sdfs=[out1, preprocessor1.destandardize(x_edited), th.abs(out1- preprocessor1.destandardize(x_edited))], 
    title = f"Optimization on the latent at $t={ddpm_sampler1.ddim_timesteps[t_optim_idx]}$ based on the volume at $t=0$",
    titles=[
        [f"Original ($V = {volume_estimates(out1_std)[i].item():.2f}$)" for i in range(out1.shape[0])], 
        [f"Edited ($V_E = {volume_estimates(x_edited)[i].item():.2f} - V_{{target}}: {volume_estimates(out1_std)[i].item() * (1+target_volume_increment):.2f})$" for i in range(out1.shape[0])],
        [f"Absolute difference between original and edited" for i in range(out1.shape[0])]
        ]
)